In [1]:
import os
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
with open('client_id.txt') as file:
    client_id = file.read()
    
with open('client_secret.txt') as file:
    client_secret = file.read()
    
with open('username.txt') as file:
    username = file.read()

In [3]:
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-library-read'

token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

In [4]:
if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
else:
    print("Can't get token for", username)

In [5]:
list_of_tracks = []

for i in range(len(tracks)):
    list_of_tracks.append(tracks[i]['track']['name'])

In [6]:
len(list_of_tracks)

407

In [7]:
list_of_artist_names = []

for i in range(len(tracks)):
    list_of_artist_names.append(tracks[i]['track']['album']['artists'][0]['name'])

In [8]:
len(list_of_artist_names)

407

In [9]:
list_of_artist_ids = []

for i in range(len(tracks)):
    list_of_artist_ids.append(tracks[i]['track']['album']['artists'][0]['id'])

In [10]:
len(list_of_artist_ids)

407

In [11]:
list_of_track_ids = []

for i in range(len(tracks)):
    list_of_track_ids.append(tracks[i]['track']['id'])

In [12]:
len(list_of_track_ids)

407

In [13]:
list_of_durations = []

for i in range(len(tracks)):
    list_of_durations.append(tracks[i]['track']['duration_ms'])

In [14]:
len(list_of_durations)

407

In [15]:
list_of_albums = []

for i in range(len(tracks)):
    list_of_albums.append(tracks[i]['track']['album']['name'])

In [16]:
len(list_of_albums)

407

In [17]:
list_of_popularities = []

for i in range(len(tracks)):
    list_of_popularities.append(tracks[i]['track']['popularity'])

In [18]:
len(list_of_popularities)

407

In [19]:
import pandas as pd

all_songs = pd.DataFrame({'artist': list_of_artist_names,
                         'artist_id': list_of_artist_ids,
                         'track_name': list_of_tracks,
                         'track_id': list_of_track_ids,
                         'duration_ms': list_of_durations,
                         'album': list_of_albums,
                         'popularity': list_of_popularities})

In [20]:
sp = spotipy.Spotify(auth=token)

In [21]:
index = 0
audio_features = []

while index < all_songs.shape[0]:
    audio_features += sp.audio_features(all_songs.iloc[index:index+50, 3])
    index += 50

In [22]:
len(audio_features)

407

In [23]:
features_list = []

for features in audio_features:
    features_list.append([features['danceability'],
                         features['energy'],
                         features['tempo'],
                         features['loudness'],
                         features['valence'],
                         features['speechiness'],
                         features['instrumentalness'],
                         features['liveness'],
                         features['acousticness']])

In [24]:
len(features_list)

407

In [25]:
df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'energy', 'tempo', 'loudness', 'valence', 'speechiness', 'instrumentalness', 'liveness', 'acousticness'])

In [26]:
songs_data = pd.concat([all_songs, df_audio_features], axis=1)

In [27]:
songs_data.to_csv('songs_data.csv', index=False)